# Obtener de la API Tweets que no sean retweet y que contengan el hashtag #SpiderMan2 en inglés.
Realizar limpieza en los datos
Evaluar la polaridad

In [19]:
import os
from dotenv import load_dotenv
# Cargar valores del archivo .env en las variables de entorno
load_dotenv()
# Cargar valor del token a variable
bearer_token = os.environ.get("BEARER_TOKEN")

url = "https://api.twitter.com/2/tweets/search/recent"
headers = {
    "Authorization": f"Bearer {bearer_token}",
    "User-Agent":"v2FullArchiveSearchPython"
}
hastag='#SpiderMan2'
params = {
    'query': f'{hastag} -is:retweet lang:en', #Va a traer los que tienen el hashtag '#machinelearning' y el -is:retweet es para que no traigra retweets
    'max_results':100  #Y que traiga 100 resultados maximo
    
}
import requests
response = requests.get(url, headers=headers, params=params)
print(response)
# Generar excepción si la respuesta no es exitosa
if response.status_code != 200:
    raise Exception(response.status_code, response.text)
import pandas as pd
df = pd.json_normalize(response.json()['data'])
df

<Response [200]>


,id,text
0,1454222643348099077,#spiderman2 credits https://t.co/CFLU8Znyf9
1,1454220771614728198,@Marvels_Spider1 hello friend how are you i ne...
2,1454203986987290630,@SamRamani2's #SpiderMan2 is still a top tier ...
3,1454200851271139329,@angelsabadbitch I'd love if @insomniacgames w...
4,1454161493335941125,Day 48 of asking Insomniac to please add the C...
...,...,...
95,1451971049617231880,Day 42 of asking Insomniac to please add the C...
96,1451969648271011844,Nostalgia hits hard 🥲 OH AND THIS GAME WAS SO ...
97,1451956585987719169,Spider-Man 2 (2004)...\nSeen the film? Let me ...
98,1451906290830102531,#SpiderMan2 Fusion Reactor 🤞🏻🙂🔥\n\n#SpiderManN...


In [20]:
from nltk.tokenize import TweetTokenizer # libreria especifica para los tokenizar tweets 

tt = TweetTokenizer() # intanciamos ya que es un modelo lo que vamos a llamar
# Aplicar Tokenizer a la columna
tokenized_text = df['text'].apply(tt.tokenize)
df["tokenized_text"] = tokenized_text
df=df[['tokenized_text']]
df

,tokenized_text
0,"[#spiderman2, credits, https://t.co/CFLU8Znyf9]"
1,"[@Marvels_Spider1, hello, friend, how, are, yo..."
2,"[@SamRamani2, ', s, #SpiderMan2, is, still, a,..."
3,"[@angelsabadbitch, I'd, love, if, @insomniacga..."
4,"[Day, 48, of, asking, Insomniac, to, please, a..."
...,...
95,"[Day, 42, of, asking, Insomniac, to, please, a..."
96,"[Nostalgia, hits, hard, 🥲, OH, AND, THIS, GAME..."
97,"[Spider-Man, 2, (, 2004, ), ..., Seen, the, fi..."
98,"[#SpiderMan2, Fusion, Reactor, 🤞, 🏻, 🙂, 🔥, #Sp..."


In [21]:
# Crear lista sin stopwords
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def clean(x):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F" # emoticons
        u"\U0001F300-\U0001F5FF" # symbols & pictographs
        u"\U0001F680-\U0001F6FF" # transport & map symbols
        u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           "]+", flags = re.UNICODE)
    n = regrex_pattern.sub(r'', x)
    if (not x.lower() in stop_words) and (x[0] != '@') and (n != ''):
        return True
    else:
        return False
for index, tweet in enumerate(tokenized_text):
    tweet = [x for x in tweet if clean(x)]
    tokenized_text[index] = tweet

df["tokenized_text"] = tokenized_text
df

<ipython-input-21-bb8f67a38a37>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tokenized_text"] = tokenized_text


,tokenized_text
0,"[#spiderman2, credits, https://t.co/CFLU8Znyf9]"
1,"[hello, friend, need, help, would, like, share..."
2,"[', #SpiderMan2, still, top, tier, sequel, ., ..."
3,"[I'd, love, would, make, Daredevil, DLC, Sony,..."
4,"[Day, 48, asking, Insomniac, please, add, Capt..."
...,...
95,"[Day, 42, asking, Insomniac, please, add, Capt..."
96,"[Nostalgia, hits, hard, 🥲, OH, GAME, GOOD, IWO..."
97,"[Spider-Man, 2, (, 2004, ), ..., Seen, film, ?..."
98,"[#SpiderMan2, Fusion, Reactor, 🤞, #SpiderManNo..."


In [24]:
#segunda limpieza
import re
i=0
for text in df['tokenized_text']:
    #text=eval(str_text) # lo convertimos en lista para poder iterar por palabra
    #print("\nTEXTO A ANALIZAR ->",text)
    j=0
    for palabra in text:
        if(re.search('^#+',palabra)):
            palabra = re.sub(r'.', '', palabra, count = 1) # eliminamos el primer caracter que es "#"
            text[j]=palabra
            j=j+1
        elif(re.search('^@+',palabra)):
            palabra = re.sub(r'.', '', palabra, count = 1) # eliminamos el primer caracter que es "@"
            text[j]=palabra
            j=j+1
        elif(re.search('^http+',palabra)):
            text.remove(palabra)  #ACA NO SUMAMOS J PORQUE QUITAMOS UN ELEMENTO DE LA LISTA
        elif(len(palabra)==1):
            text.remove(palabra)  #ACA NO SUMAMOS J PORQUE QUITAMOS UN ELEMENTO DE LA LISTA
        else:
            text[j]=palabra
            j=j+1
    #print("\n TEXTO DEVULETO ->", text) 
    df['tokenized_text'][i] = text
    i=i+1
df

/media/eduardo/Anaconda/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,tokenized_text
0,"[spiderman2, credits]"
1,"[hello, friend, need, help, would, like, share..."
2,"[still, top, tier, sequel, movie, along, super..."
3,"[I'd, love, would, make, Daredevil, DLC, Sony,..."
4,"[Day, 48, asking, Insomniac, please, add, Capt..."
...,...
95,"[Day, 42, asking, Insomniac, please, add, Capt..."
96,"[Nostalgia, hits, hard, GAME, GOOD, IWONT, LET..."
97,"[Spider-Man, 2004, Seen, film, know, favourite..."
98,"[SpiderMan2, Fusion, Reactor, SpiderManNoWayHome]"
